In [33]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

# Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [55]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [56]:
#2. 문자 집합 만들기
world_set = list(set(sentence))
# set(sentence) -> sentence에 있는 unique한 것들만 random order로 남겨 {}로 둘러싸인 
# set object로 return, 이를 list로 만들어 world_set에 할당

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {token: i for i, token in enumerate(world_set)}

In [57]:
print(vocab)

{' ': 0, 'f': 1, 'n': 2, 's': 3, 'd': 4, 'g': 5, 't': 6, "'": 7, 'u': 8, 'l': 9, 'k': 10, 'a': 11, 'm': 12, 'b': 13, 'e': 14, 'c': 15, 'i': 16, ',': 17, 'w': 18, 'p': 19, 'o': 20, 'y': 21, '.': 22, 'h': 23, 'r': 24, 'B': 25}


In [58]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [59]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [60]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : (sequence_length + i)]
  y_str = sentence[(i + 1) : (sequence_length + i + 1)]
  print(i, x_str, "->", y_str)

  # x_str과 y_str의 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [61]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[25, 24, 16, 15, 10, 0, 18, 11, 9, 9]
[24, 16, 15, 10, 0, 18, 11, 9, 9, 3]


In [62]:
len(x_data)

188

In [63]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(len(vocab))[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data) # cross entropy loss expects its label to be long tensor type

In [64]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape)) 
# 188개의 data에 대해 한 data당 10개의 단어에 해당하는 길이 26의 vector
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [65]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [66]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([24, 16, 15, 10,  0, 18, 11,  9,  9,  3])


In [67]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.layers = layers
    self.rnn = torch.nn.RNN(input_size = input_dim, hidden_size = hidden_dim,
                            num_layers = layers)
    self.fc = torch.nn.Linear(hidden_dim, 26)
  
  def forward(self, x):
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [68]:
net = Net(vocab_size, hidden_size, 2)

In [69]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [70]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [71]:
Y.shape

torch.Size([188, 10])

In [72]:
b = outputs.view(-1, 26)
b.shape

torch.Size([1880, 26])

In [73]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, 26), Y.view(1880))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]
    print(predict_str)

..............r..r..r.r......r...r......r...... r..r.r.r...r.. .r................rr..r.r..r..r.r.......r..r....r...r....r..r........r.. .r................r..r..r..r.........r......r.....r.r. .r.r..
 r.r  rrrr                                                                                                                                                                                           
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [74]:
outputs = net(X)
results = outputs.argmax(dim=2)
[world_set[t] for t in result]

['n', 't', ' ', 't', 'h', 'i', 'n', 'g', 's', 'c']

In [75]:
predict_str

"reck b lss are there for a reason and you must not think that the brick walls aren't there to keep us out, but rather in this way that the brick walls are there to show us how badly we want thingsc"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

# Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요! 

영어가 아닌 한국어로 시도해보는 것도 좋겠죠? 

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

## 영어

예제 구문의 brick walls를 보고 Pink Floyd의 Another Brick In The Wall이라는 노래가 생각이 나서 해당 노래의 가사를 가지고 왔습니다.

In [76]:
#1. 생성할 문장 데이터

sentence = ("We don't need no education "
            "We don't need no thought control "
            "No dark sarcasm in the classroom "
            "Teachers, leave them kids alone "
            "Hey! Teacher! Leave us kids alone!")

In [77]:
#2. 문자 집합 만들기
world_set = list(set(sentence))
# set(sentence) -> sentence에 있는 unique한 것들만 random order로 남겨 {}로 둘러싸인 
# set object로 return, 이를 list로 만들어 world_set에 할당

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
vocab = {token: i for i, token in enumerate(world_set)}

In [78]:
print(vocab)

{' ': 0, 'n': 1, 's': 2, 'd': 3, 'g': 4, 't': 5, "'": 6, 'u': 7, 'H': 8, 'l': 9, 'W': 10, 'k': 11, 'a': 12, 'm': 13, '!': 14, 'e': 15, 'L': 16, 'c': 17, 'i': 18, ',': 19, 'N': 20, 'v': 21, 'o': 22, 'T': 23, 'y': 24, 'h': 25, 'r': 26}


In [79]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 27


In [80]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!)

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 8  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [81]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i : (sequence_length + i)]
  y_str = sentence[(i + 1) : (sequence_length + i + 1)]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 We don't -> e don't 
1 e don't  ->  don't n
2  don't n -> don't ne
3 don't ne -> on't nee
4 on't nee -> n't need
5 n't need -> 't need 
6 't need  -> t need n
7 t need n ->  need no
8  need no -> need no 
9 need no  -> eed no e
10 eed no e -> ed no ed
11 ed no ed -> d no edu
12 d no edu ->  no educ
13  no educ -> no educa
14 no educa -> o educat
15 o educat ->  educati
16  educati -> educatio
17 educatio -> ducation
18 ducation -> ucation 
19 ucation  -> cation W
20 cation W -> ation We
21 ation We -> tion We 
22 tion We  -> ion We d
23 ion We d -> on We do
24 on We do -> n We don
25 n We don ->  We don'
26  We don' -> We don't
27 We don't -> e don't 
28 e don't  ->  don't n
29  don't n -> don't ne
30 don't ne -> on't nee
31 on't nee -> n't need
32 n't need -> 't need 
33 't need  -> t need n
34 t need n ->  need no
35  need no -> need no 
36 need no  -> eed no t
37 eed no t -> ed no th
38 ed no th -> d no tho
39 d no tho ->  no thou
40  no thou -> no thoug
41 no thoug -> o though
42

In [82]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

print(x_data[0])
print(y_data[0])

[10, 15, 0, 3, 22, 1, 6, 5]
[15, 0, 3, 22, 1, 6, 5, 0]


In [83]:
len(x_data)

151

In [84]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(len(vocab))[x] for x in x_data]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data) # cross entropy loss expects its label to be long tensor type

In [85]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape)) 
# 188개의 data에 대해 한 data당 10개의 단어에 해당하는 길이 26의 vector
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([151, 8, 27])
레이블의 크기 : torch.Size([151, 8])


In [86]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0

In [87]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([15,  0,  3, 22,  1,  6,  5,  0])


In [88]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.layers = layers
    self.rnn = torch.nn.RNN(input_size = input_dim, hidden_size = hidden_dim,
                            num_layers = layers)
    self.fc = torch.nn.Linear(hidden_dim, 27)
  
  def forward(self, x):
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [89]:
net = Net(vocab_size, hidden_size, 2)

In [90]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [91]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([151, 8, 27])


In [92]:
Y.shape

torch.Size([151, 8])

In [93]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, 27), Y.view(151 * 8))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]
    print(predict_str)

seeessTsee,,,,,,e,,,,,,W,,,ee,,,,,,ee,,,,,,eee,s,e,e,,,,W,ee,se,,,,e,T,,,,,,We,,W,,,,,,,,,W,ese,,,T,,,,ee,,W,eWW,eee,,,,,e,,,,,,W,W,,,,T,,eW,,,a,,,a,e,,,,,e,,
eeeeeeseeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee
eeeeeeeeeee                                                                                                                                                   
eeeeee eee                                    

In [94]:
predict_str

"e don't need no education We don't need no thought control No dark sarcasm in the classroom Teachers, leave them kids alone Hey! Teacher! Leave us kids aloner"

큰 수정 없이 sequence length와 shape만 조금 바꿔줘어서 돌렸는데 크게 나쁘지는 않은 결과가 나온 것 같다.

## 한국어

위에서 사용한 가사를 번역해서 사용

In [5]:
sentence = ("우리는 교육이 필요 없습니다 "
            "우리는 사고 통제가 필요하지 않습니다 "
            "교실에서 어두운 풍자 금지 "
            "선생님, 아이들은 그냥 두세요 "
            "이봐! 선생님! 아이들을 그냥 내버려 두세요!")

In [96]:
type(sentence)

str

띄어쓰기만으로도 어느 정도의 토큰화가 가능한 영어와는 달리, 조사, 어미 등이 존재하는 한국어는 형태소 단위로 토큰화를 진행해주어야한다고 한다. 따라서 한국어 토큰화를 위해 한국어 자연어처리를 위한 패키지인 KoNLPy의 다양한 클래스를 사용해보았다.

In [97]:
!pip3 install konlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [98]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [99]:
# import os
# os.chdir('/content/drive/MyDrive/Colab Notebooks')

#### 한나눔

In [100]:
from konlpy.tag import Hannanum
hannanum=Hannanum()

In [101]:
print(hannanum.morphs(sentence))

['우리', '는', '교육', '이', '필요', '없', '습니다', '우리', '는', '사', '고', '통제', '가', '필요', '하', '지', '않', '습니다', '교실', '에서', '어둡', '은', '풍자', '금지', '선생님', ',', '아이들', '은', '그냥', '두', '이', '세', '요', '이봐', '!', '선생님', '!', '아이들', '을', '그냥', '내', '어', '버리', '어', '두', '세', '요', '!']


사고를 동사 사고로 인식해서 사 + 고로 분리한 듯 하다.  
아이 + 들이 아닌 아이들로 토큰화했다.  
선생 + 님이 아닌 선생님으로 토큰화했다.
두세요를 두 + 시 + 어요가 아닌 두 + 이 + 세 + 요, 두 + 세 + 요 등 서로 다른 잘못된 방법으로 분리하고 있다.  
내버려두다를 내버리 + 어 + 두다가 아닌 내 + 어 + 버리 + 어 + 두로 분리하고 있다.


#### 꼬꼬마

In [102]:
from konlpy.tag import Kkma
kkma=Kkma()

In [103]:
print(kkma.morphs(sentence))

['우리', '는', '교육', '이', '필요', '없', '습니다', '우리', '는', '사고', '통제', '가', '필요', '하', '지', '않', '습니다', '교실', '에서', '어둡', 'ㄴ', '풍자', '금지', '선생님', ',', '아이', '들', '은', '그냥', '두', '세요', '이봐', '!', '선생님', '!', '아이', '들', '을', '그냥', '내버리', '어', '두', '세요', '!']


한나눔과는 다르게 사고를 사 + 고가 아닌 사고로 잘 분리했다.  
아이들도 아이 + 들로 잘 토큰화했다.  
선생님은 여전히 선생 + 님이 아닌 선생님으로 분리되어있다.
한나눔은 어두운을 어둡 + 은으로 잘 분리했으나 꼬꼬마에서는 어둡 + ㄴ으로 분리하고 있다.  
두세요는 두 + 시 + 어요가 아닌 두+세요로 분리되어있다.   
내버려두다는 내버리 + 어 + 두다로 잘 분리되어있다.


#### 코모란

In [104]:
from konlpy.tag import Komoran
komoran=Komoran()

In [53]:
print(komoran.morphs(sentence))

['우리', '는', '교육', '이', '필요', '없', '습니다', '우리', '는', '사고', '통제', '가', '필요', '하', '지', '않', '습니다', '교실', '에서', '어둡', 'ㄴ', '풍자', '금지', '선생님', ',', '아이들', '은', '그냥', '두', '시', '어요', '이봐', '!', '선생님', '!', '아이', '들', '을', '그냥', '내버리', '어', '두', '시', '어요', '!']


어두운이 꼬꼬마와 마찬가지로 어둡 + ㄴ 으로 분리되어있다.  
선셍님도 선생 + 님이 아닌 선생님으로 분리되어있다.  
뒤에 나오는 아이들은 아이 + 들로 잘 분리되어있지만 처음 나오는 아이들은 한나눔 때와 마찬가지로 아이들로 토큰화되었다.  
두세요의 경우 두 + 시 + 어요로 잘 분리되어있다.

#### Okt

In [105]:
from konlpy.tag import Okt
okt=Okt()

In [106]:
print(okt.morphs(sentence))

['우리', '는', '교육', '이', '필요', '없습니다', '우리', '는', '사고', '통제', '가', '필요하지', '않습니다', '교실', '에서', '어', '두운', '풍자', '금지', '선생님', ',', '아이', '들', '은', '그냥', '두세요', '이', '봐', '!', '선생님', '!', '아이', '들', '을', '그냥', '내버려', '두세요', '!']


언뜻 보기에도 형태소 분리가 이루어지지 않은 단어들이 많아 위와 같은 주석은 안 달았음

#### Mecab

In [ ]:
# 코랩용 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

In [2]:
from konlpy.tag import Mecab
mecab = Mecab()
# 런타임 재시작 안하면 계속 install이 안됐다고 오류 뜸

In [3]:
print(mecab.morphs(sentence))

['우리', '는', '교육', '이', '필요', '없', '습니다', '우리', '는', '사고', '통제', '가', '필요', '하', '지', '않', '습니다', '교실', '에서', '어두운', '풍자', '금지', '선생', '님', ',', '아이', '들', '은', '그냥', '두', '세요', '이봐', '!', '선생', '님', '!', '아이', '들', '을', '그냥', '내버려', '두', '세요', '!']


어두운이 형태소 분리가 안돼있음  
선생님이 처음으로 선생 + 님으로 잘 분리가 되었다.  
내버려가 내버리 + 어로 분리가 안돼있음
두세요도 두 + 시 + 어요 로 분리가 안돼있음

### 토큰화

패키지들의 성능이 엇비슷하지만 그래도 내 경우의 가장 성능이 좋았다고 생각되는 코모란을 샤용하겠음

In [3]:
from konlpy.tag import Komoran
komoran = Komoran()

In [6]:
wordset = komoran.morphs(sentence)

In [7]:
print(wordset)

['우리', '는', '교육', '이', '필요', '없', '습니다', '우리', '는', '사고', '통제', '가', '필요', '하', '지', '않', '습니다', '교실', '에서', '어둡', 'ㄴ', '풍자', '금지', '선생님', ',', '아이들', '은', '그냥', '두', '시', '어요', '이봐', '!', '선생님', '!', '아이', '들', '을', '그냥', '내버리', '어', '두', '시', '어요', '!']


In [8]:
wordset[20] = '은'

for i in range(len(wordset)):
  if wordset[i] == "선생님":
    wordset[i] = "선생"
    wordset.insert(i + 1, "님")

for i in range(len(wordset)):
  if wordset[i] == "아이들":
    wordset[i] = "아이"
    wordset.insert(i + 1, "들")

wordset.append(' ')

코모란을 이용한 후 부족하다고 생각되는 부분들은 직접 바꿔주거나 분리해주었음

In [9]:
print(wordset)

['우리', '는', '교육', '이', '필요', '없', '습니다', '우리', '는', '사고', '통제', '가', '필요', '하', '지', '않', '습니다', '교실', '에서', '어둡', '은', '풍자', '금지', '선생', '님', ',', '아이', '들', '은', '그냥', '두', '시', '어요', '이봐', '!', '선생', '님', '!', '아이', '들', '을', '그냥', '내버리', '어', '두', '시', '어요', '!', ' ']


In [12]:
# u_wordset = list(set(wordset))
u_wordset = list(pd.unique(wordset))

input의 unique한 값들을 랜덤한 순서로 return하는 set을 쓸 때보다 원래 가지고 있던 고유한 순서대로 return하는 pd.unique를 썼을 때가 더 그럴듯한 문장이 예측되었음.  
우연에 의한 것인지, 아니면 일부러 무작위성 부여를 위해 set을 쓰는 거라서 이런 방법을 쓰면 안되는 건지 모르겠지만 우선 이렇게 진행함

In [13]:
vocab = {token: i for i, token in enumerate(u_wordset)}
print(vocab)

{'우리': 0, '는': 1, '교육': 2, '이': 3, '필요': 4, '없': 5, '습니다': 6, '사고': 7, '통제': 8, '가': 9, '하': 10, '지': 11, '않': 12, '교실': 13, '에서': 14, '어둡': 15, '은': 16, '풍자': 17, '금지': 18, '선생': 19, '님': 20, ',': 21, '아이': 22, '들': 23, '그냥': 24, '두': 25, '시': 26, '어요': 27, '이봐': 28, '!': 29, '을': 30, '내버리': 31, '어': 32, ' ': 33}


In [14]:
len(vocab)

34

### Sequence 길이 단위 자르기

In [ ]:
# #5. seqence 길이 단위 자르기

# # 데이터 구성을 위한 리스트
# x_data = []
# y_data = []

# ## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

# for i in range(0, len(sentence) - sequence_length):
#   x_str = sentence[i : (sequence_length + i)]
#   y_str = sentence[(i + 1) : (sequence_length + i + 1)]
#   print(i, x_str, "->", y_str)

#   # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
#   x_data.append([vocab[c] for c in x_str])
#   y_data.append([vocab[d] for d in y_str])

위의 코드를 그대로 사용했을 경우에 발생하는 문제



*   x_str = sentence[i : (sequence_length + i)]:  
현재 나는 형태소 단위의 sequencing을 하고자 했는데 이 방식을 사용 시, 그냥 길이에 맞게끔 단어가 중간에 잘려버리는 문제가 발생함 
*   x_data.append([vocab[c] for c in x_str]):  
이 역시 형태소 한 단위단위가 매 루프마다 추가되길 원하지만 이 코드를 그대로 사용했을 시, 한 글자씩 잘려서 들어가게 되기 때문에(예를 들어 '우리'가 아닌 '우'를 추가하려고 함) vocab에 해당 단어가 존재하지 않아 오류를 발생시키게 된다.  


따라서 아래와 같은 방법으로 위 코드의 기능을 수행함



In [15]:
x = wordset[:-1]
y = wordset[1:]

In [16]:
x_voc = []
y_voc = []

for c in x:
  for key, value in vocab.items():
    if key == c:
      x_voc.append(value)

for d in y:
  for key, value in vocab.items():
    if key == d:
      y_voc.append(value)

In [17]:
vocab_size = len(vocab)
hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 8  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [18]:
x_seq = []
y_seq = []

for i in range(len(x_voc) - sequence_length + 1):
  x_seq.append(x_voc[i: i + sequence_length])
  y_seq.append(y_voc[i: i + sequence_length])

In [19]:
print(x_seq)
print(y_seq)

[[0, 1, 2, 3, 4, 5, 6, 0], [1, 2, 3, 4, 5, 6, 0, 1], [2, 3, 4, 5, 6, 0, 1, 7], [3, 4, 5, 6, 0, 1, 7, 8], [4, 5, 6, 0, 1, 7, 8, 9], [5, 6, 0, 1, 7, 8, 9, 4], [6, 0, 1, 7, 8, 9, 4, 10], [0, 1, 7, 8, 9, 4, 10, 11], [1, 7, 8, 9, 4, 10, 11, 12], [7, 8, 9, 4, 10, 11, 12, 6], [8, 9, 4, 10, 11, 12, 6, 13], [9, 4, 10, 11, 12, 6, 13, 14], [4, 10, 11, 12, 6, 13, 14, 15], [10, 11, 12, 6, 13, 14, 15, 16], [11, 12, 6, 13, 14, 15, 16, 17], [12, 6, 13, 14, 15, 16, 17, 18], [6, 13, 14, 15, 16, 17, 18, 19], [13, 14, 15, 16, 17, 18, 19, 20], [14, 15, 16, 17, 18, 19, 20, 21], [15, 16, 17, 18, 19, 20, 21, 22], [16, 17, 18, 19, 20, 21, 22, 23], [17, 18, 19, 20, 21, 22, 23, 16], [18, 19, 20, 21, 22, 23, 16, 24], [19, 20, 21, 22, 23, 16, 24, 25], [20, 21, 22, 23, 16, 24, 25, 26], [21, 22, 23, 16, 24, 25, 26, 27], [22, 23, 16, 24, 25, 26, 27, 28], [23, 16, 24, 25, 26, 27, 28, 29], [16, 24, 25, 26, 27, 28, 29, 19], [24, 25, 26, 27, 28, 29, 19, 20], [25, 26, 27, 28, 29, 19, 20, 29], [26, 27, 28, 29, 19, 20, 29, 

### 원핫인코딩

In [25]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?
x_one_hot = [np.eye(len(vocab))[x] for x in x_seq]

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_seq) # cross entropy loss expects its label to be long tensor type

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  if __name__ == '__main__':


In [26]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape)) 
# 188개의 data에 대해 한 data당 10개의 단어에 해당하는 길이 26의 vector
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([41, 8, 34])
레이블의 크기 : torch.Size([41, 8])


In [27]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [28]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([1, 2, 3, 4, 5, 6, 0, 1])


### 모델

In [29]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_fisrt 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim
    self.layers = layers
    self.rnn = torch.nn.RNN(input_size = input_dim, hidden_size = hidden_dim,
                            num_layers = layers)
    self.fc = torch.nn.Linear(hidden_dim, 34)
  
  def forward(self, x):
    x, hidden = self.rnn(x)
    x = self.fc(x)
    return x

In [30]:
net = Net(vocab_size, hidden_size, 2)

In [34]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [35]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([41, 8, 34])


In [36]:
Y.shape

torch.Size([41, 8])

In [37]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    loss = criterion(outputs.view(-1, 34), Y.view(41 * 8))
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([wordset[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += wordset[result[-1]]
    print(predict_str)

필요어요어요않않않않필요이필요필요않가않않않통제않통제어요가,않않통제않않어요통제이않통제않통제통제않통제사고통제어요통제통제않통제않통제않통제
필요은은은않은은필요,은은은은사고은은은사고은은사고은은은사고사고사고어둡사고사고은사고사고은사고사고않사고사고어둡은은은은은어둡사고습니다
은은은은은은은은,은은은은습니다습니다은어둡습니다습니다습니다어둡습니다은은습니다어둡금지습니다금지습니다은어둡어둡습니다어둡습니다습니다금지습니다습니다금지습니다은은은어둡습니다습니다
은은은은은은은은,은,은은선생습니다은어둡습니다습니다습니다어둡습니다선생은선생금지선생습니다선생습니다은들선생습니다어둡선생습니다금지선생습니다금지습니다선생,은어둡선생습니다
은은은은은은은은,은,은은선생그냥은어둡선생습니다습니다어둡습니다어둡은선생금지선생습니다님,은들선생그냥어둡선생그냥금지선생습니다님,님,은들선생그냥
은은은은은습니다은은,은,은들아이그냥은하선생그냥습니다어둡그냥그냥은그냥금지선생습니다님,아이들아이그냥어둡은그냥금지선생습니다님,님,아이들아이그냥
는은은은은습니다은는,는,필요들가그냥습니다하선생그냥습니다어둡그냥그냥은그냥금지선생습니다님,아이들아이그냥어둡은그냥금지선생습니다님,님,아이들아이그냥
는은은은은습니다은는우리는,필요통제가그냥습니다하은않습니다교실그냥어둡은그냥금지선생습니다님,아이들은그냥어둡은그냥금지선생습니다님,님,아이들은그냥
는은은필요은습니다습니다는우리는,필요통제가그냥습니다하지않습니다교실그냥어둡은그냥금지선생습니다님,아이들은그냥어둡선생그냥금지선생습니다님,아이,아이들은그냥
는우리은필요은습니다습니다는우리는우리필요통제가필요습니다하지않습니다교실그냥어둡은그냥금지선생습니다님,아이들은그냥어둡은그냥금지선생습니다님,아이,아이들은그냥
는우리는필요은습니다습니다는우리는우리필요통제가필요습니다하지않습니다교실그냥어둡은풍자금지선생습니다님,아이들은그냥어둡은그냥금지선생습니다님,아이,아이들은그냥
는우리는필요은습니다습니다는우리는우리필요통제가필요습니다하지않습니다교실그냥어둡은풍자금지선생습니다님,아이들은그냥어둡은그냥금지선생습니다님,아이,아이들은그냥
는우리는필요은습니다습니다는우리는우리필요통제가필요습니다하지

In [38]:
predict_str

'는교육이필요없습니다우리는우리는사고필요통제가필요습니다하지않습니다교실에서어둡은풍자금지선생습니다님,아이들은그냥어둡은그냥금지선생두님시어요,아이들그냥이봐'

문장이 그럴 듯하지는 못하지만 해보았다는 것에 의의!

### PyKoSpacing을 이용한 띄어쓰기 해보기

In [ ]:
pip install git+https://github.com/haven-jeon/PyKoSpacing.git

In [39]:
from pykospacing import Spacing
spacing = Spacing()

In [40]:
prd_str_sp = spacing(predict_str)
print(prd_str_sp)

는 교육이 필요 없습니다 우리는 우리는 사고 필요 통제가 필요습니다 하지 않습니다교실에서 어둡은 풍자금지 선생습니다님, 아이들은 그냥 어둡은 그냥 금지 선생 두님시어요, 아이들 그냥이 봐
